In [1]:
import pandas as pd

In [2]:
diabetes = pd.read_csv('diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
x = diabetes.drop('Outcome',axis=1)
y = diabetes['Outcome']

In [6]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 48.1 KB


In [16]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

x = scale.fit_transform(x)

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
x_test.shape

(154, 8)

### Training a simple modle without using keras_tuner

In [19]:
import tensorflow as tf 
from tensorflow import keras

In [20]:
model = keras.Sequential()

model.add(keras.layers.Dense(64,activation='relu',input_shape=(8,)))
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.summary()

F:\Data Science\Deep Learning\deep_learning\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### We can see that model is overfitting as the validation accuracy is constant however accuracy increasing

In [23]:
model.fit(x_train,y_train,epochs=100,batch_size=8,validation_split=0.1)

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.7168 - loss: 0.6026 - val_accuracy: 0.6774 - val_loss: 0.5887
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7017 - loss: 0.5457 - val_accuracy: 0.7419 - val_loss: 0.5383
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7535 - loss: 0.4822 - val_accuracy: 0.7581 - val_loss: 0.5152
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7768 - loss: 0.4499 - val_accuracy: 0.7581 - val_loss: 0.5018
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7921 - loss: 0.4438 - val_accuracy: 0.7581 - val_loss: 0.4902
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7667 - loss: 0.4556 - val_accuracy: 0.7581 - val_loss: 0.4863
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7838 - loss: 0.4268 - val_accuracy: 0.7742 - val_loss: 0.4844
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7863 - loss: 0.4345 - val_accuracy: 0.7742 - 

In [24]:
model.evaluate(x_test,y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7429 - loss: 0.6007 


[0.6323383450508118, 0.7337662577629089]

### Using keras tuner
- Finding best hyperparameter for the model

In [26]:
import keras_tuner as kt

#### Finding the best optimizer

In [59]:
def build_model(hp):
    model1 = keras.Sequential()

    model1.add(keras.layers.Dense(64,activation='relu',input_shape=(8,)))
    model1.add(keras.layers.Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values = ['sgd', 'rmsprop', 'adam', 'adamw', 'adadelta'])

    model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model1

In [62]:
tuner = kt.RandomSearch(build_model, objective='accuracy', max_trials=5, directory='mydir', project_name='optimizer')

F:\Data Science\Deep Learning\deep_learning\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
tuner.search(x_train, y_train, epochs=5,validation_data=(x_test,y_test), verbose=1)

Trial 5 Complete [00h 00m 03s]
accuracy: 0.37622150778770447

Best accuracy So Far: 0.7687296271324158
Total elapsed time: 00h 00m 15s


In [66]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [67]:
keras_model = tuner.get_best_models(num_models=1)[0]

In [68]:
keras_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
keras_model.fit(x_train,y_train,epochs=20, validation_split=0.1)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7491 - loss: 0.5039 - val_accuracy: 0.7419 - val_loss: 0.5049
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8154 - loss: 0.4317 - val_accuracy: 0.7258 - val_loss: 0.4983
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7933 - loss: 0.4571 - val_accuracy: 0.7419 - val_loss: 0.4926
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7897 - loss: 0.4627 - val_accuracy: 0.7419 - val_loss: 0.4877
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7883 - loss: 0.4449 - val_accuracy: 0.7419 - val_loss: 0.4831
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7956 - loss: 0.4307 - val_accuracy: 0.7419 - val_loss: 0.4815
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7833 - loss: 0.4386 - val_accuracy: 0.7419 - val_loss: 0.4808
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8043 - loss: 0.4366 - val_accuracy: 0.7258 - val_loss

In [69]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'rmsprop', 'adam', 'adamw', 'adadelta'], 'ordered': False}


In [99]:
tuner.search(x_train,y_train,epochs=5, validation_data=(x_test,y_test))
for trial in tuner.oracle.get_best_trials(num_trials=6):
    print('id', trial.trial_id)
    print(trial.hyperparameters.values)
    print(trial.score)

id 1
{'optimizer': 'adam'}
0.7687296271324158
id 3
{'optimizer': 'rmsprop'}
0.7638436555862427
id 2
{'optimizer': 'adamw'}
0.7524430155754089
id 0
{'optimizer': 'sgd'}
0.7149837017059326
id 4
{'optimizer': 'adadelta'}
0.37622150778770447


In [94]:
tuner.results_summary()

Results summary
Results in mydir\optimizer
Showing 10 best trials
Objective(name="accuracy", direction="max")

Trial 1 summary
Hyperparameters:
optimizer: adam
Score: 0.7687296271324158

Trial 3 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7638436555862427

Trial 2 summary
Hyperparameters:
optimizer: adamw
Score: 0.7524430155754089

Trial 0 summary
Hyperparameters:
optimizer: sgd
Score: 0.7149837017059326

Trial 4 summary
Hyperparameters:
optimizer: adadelta
Score: 0.37622150778770447


#### Finding Best number of layers with activation and optimizer

In [133]:
def build_model_layers(hp):
    model1 = keras.Sequential()

    layers = hp.Int('num_layers', min_value=1, max_value=10, step=2)

    for i in range(layers):
        units=hp.Int(f'units_{i}', min_value=64, max_value=1024, step=32)

        activation = hp.Choice(f'activation_{i}', values=['sigmoid', 'relu', 'tanh'])

        if i==0:
            model1.add(keras.layers.Dense(units,activation=activation, input_shape=(8,)))
        else:
            model1.add(keras.layers.Dense(units,activation=activation))
            
    optimizer = hp.Choice('optimizer', values = ['sgd', 'rmsprop', 'adam', 'adamw', 'adadelta'])

    model1.add(keras.layers.Dense(1,activation='sigmoid'))
    
    model1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model1

In [134]:
tuner1 = kt.RandomSearch(build_model_layers, objective='accuracy', max_trials=6,directory='mydir', project_name='layers')

F:\Data Science\Deep Learning\deep_learning\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [135]:
tuner1.search(x_train,y_train,epochs=100, validation_split=0.2)

Trial 6 Complete [00h 00m 36s]
accuracy: 1.0

Best accuracy So Far: 1.0
Total elapsed time: 00h 50m 07s


In [131]:
tuner1.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 2, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
activation_0 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'rmsprop', 'adam', 'adamw', 'adadelta'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
activation_1 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
activation_2 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tan

In [132]:
tuner1.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units_0': 224,
 'activation_0': 'sigmoid',
 'optimizer': 'adamw',
 'units_1': 96,
 'activation_1': 'sigmoid',
 'units_2': 288,
 'activation_2': 'tanh',
 'units_3': 224,
 'activation_3': 'tanh',
 'units_4': 576,
 'activation_4': 'sigmoid',
 'units_5': 448,
 'activation_5': 'relu',
 'units_6': 896,
 'activation_6': 'tanh'}

In [115]:
tuner1_best = tuner1.get_best_models(num_models=1)[0]

In [116]:
tuner1_best.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 224)                 │           2,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,241 (8.75 KB)

 Trainable params: 2,241 (8.75 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
tuner1_best.optimizer